# Problema - Regressão

Este é um teste para avaliar o entendimento sobre regressão. Deverá conseguir predizer as classes no problema abaixo.

Linguagem: **Python**

**Escopo do Projeto**

* O teste deverá ser feito em python;
* O conjunto de dados a ser usado esta disponivel no seguinte link: https://archive.ics.uci.edu/ml/machine-learning-databases/00288/;
* Leia o arquivo README para compreender mais sobre o conjunto de dados;
* O resultado poderá ser apresentado em html, pdf ou jupyter notebook;
* Atente-se que o resultado será avaliado também por pessoas não-técnicas. Então seja enfático e claro na explicação e nos resultados do seu trabalho;
* Lembre-se de remover a coluna com o nome da espécie da folha ao fazer o modelo não-supervisionado.

**Critérios de Avaliação:**

* Construa um modelo de regressão e responda:
    * Por que não podemos utilizar um modelo de regressão linear?
    * Qual métrica de avaliação foi utilizada e por que? Justifique.
    * Você acredita que se realizar a seleção de atributos, o seu resultado será melhor? Justifique.

O primeiro passo para a resolução do problema é a importação de todas as bibliotecas necessárias.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


Em seguida é realizada a leitura do arquivo com os dados. Como o arquivo **leaf.csv** não contêm um cabeçalho fez necessário adicionar um manualmente. Os títulos das colunas foram obtidos a partir do conteúdo do arquivo **README**

In [32]:
columns_names = ['class_species', 'specimen_number', 'eccentricity', 'aspect_ratio',
'elongation', 'solidity', 'stochastic_convexity', 'isoperimetric_factor',
'maximal_indentation_depth', 'lobedness', 'average_intensity', 'average_contrast',
'smoothness', 'third_moment', 'uniformity', 'entropy']
leafs = pd.read_csv('leaf.csv', header=None, names=columns_names)

Separa-se então o conjunto de dados em dois subgrupos: **X** e **y**. 

As variáveis independentes **X** que serviram de preditores da variável dependente **y**.

A variável _specimen_number_ foi removida do conjunto de variáveis independentes por ser uma variável categorica assim como a _class_species_.

In [50]:
X = leafs.drop(['class_species', 'specimen_number'], axis=1)
y = leafs['class_species']

A seguir é aplicado o comando describe para ter-se uma noção sobre o conjunto de dados em análise.

In [35]:
X.describe()

,eccentricity,aspect_ratio,elongation,solidity,stochastic_convexity,isoperimetric_factor,maximal_indentation_depth,lobedness,average_intensity,average_contrast,smoothness,third_moment,uniformity,entropy
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000
mean,0.719854,2.440210,0.513760,0.904158,0.943793,0.531234,0.037345,0.523845,0.051346,0.124535,0.017670,0.005928,0.000387,1.162630
std,0.208311,2.599043,0.195583,0.114639,0.115047,0.217532,0.038575,1.039639,0.035965,0.051860,0.013755,0.005294,0.000431,0.584854
min,0.117080,1.006600,0.107610,0.485490,0.396490,0.078376,0.002837,0.001464,0.005022,0.033415,0.001115,0.000229,0.000007,0.169400
25%,0.550623,1.211300,0.349623,0.890667,0.966230,0.346818,0.009521,0.016500,0.022843,0.083362,0.006901,0.002080,0.000102,0.718900
50%,0.763450,1.570750,0.501855,0.948130,0.992980,0.579160,0.023860,0.103615,0.042087,0.119375,0.014050,0.004447,0.000239,1.077450
75%,0.895097,2.343100,0.633373,0.976897,1.000000,0.700713,0.047834,0.416432,0.073046,0.163795,0.026127,0.008307,0.000516,1.554575
max,0.998710,19.038000,0.948340,0.993880,1.000000,0.858160,0.198980,7.206200,0.190670,0.280810,0.073089,0.029786,0.002936,2.708500


A maioria das variáveis têm valores entre 0 e 1, o que é bom para o modelo knn.

Apenas as variáveis: _aspect_ratio_, _lobedness_ e entropy tem valores superiores a 1. O padronização destas variáveis talvez possa melhorar a performance do modelo.

Um próximo passo importante é verificar a presença de falhas na série de dados. Como pode-se chegar na célula abaixo o conjunto de dados não apresenta falhas.

In [40]:
X.isnull().sum()

eccentricity                 0
aspect_ratio                 0
elongation                   0
solidity                     0
stochastic_convexity         0
isoperimetric_factor         0
maximal_indentation_depth    0
lobedness                    0
average_intensity            0
average_contrast             0
smoothness                   0
third_moment                 0
uniformity                   0
entropy                      0
dtype: int64

O passo seguinte então é dividir o conjunto de dados em dois grupos. O grupo de treino e o grupo de teste.

O grupo de treino será utilizado para treinar o modelo e o de teste para validar o desempenho do método.

Utilizou-se uma amostra de 70% para o treinamento do modelo e uma de 30% para a validação do modelo.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

Como este problema envolve um variável _y_ que depende de um conjunto de outras variáveis não se pode aplicar um modelo de regressão linear simples, pois, a relação entre as variáveis dependentes e a dependente não será de forma linear. Outro motivo para não se utilizar uma regressão linear simples é que o problema trata-se de uma questão de classificação. Assim, optou-se por utilizar o método dos k-vizinhos (knn). Neste método uma determinada amostra é classificada como pertecente a um determinado grupo de acordo com o valor de seus vizinhos.

A medida de desempenho utilizada foi o teste de acurácia que mede a eficiência do modelo de acordo com a quantidade de classificações corretas que ele obteve. o valor de acurácia pode variar entre 0 (o modelo classifica erroneamente todos os resultados previstos) a 1 (o modelo classifica corretamente todos os resultados previstos). Um modelo com acurácia de 0,5 é considerado como aleatório, modelos com valores superiores a 0,5 são considerados bons e menores que 0,5 ruins.

O número de n_neighbors (vizinhos) influencia no desempenho do modelo. Assim, foram testados diferentes valores de n_neighbors.

In [46]:
for k in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    y_predict = knn.predict(x_test)
    ac = accuracy_score(y_test, y_predict)
    print(f'{k} accurácia = {ac:.2f}')

1 accurácia = 0.60
2 accurácia = 0.50
3 accurácia = 0.57
4 accurácia = 0.57
5 accurácia = 0.55
6 accurácia = 0.52
7 accurácia = 0.49
8 accurácia = 0.50
9 accurácia = 0.48
10 accurácia = 0.49
11 accurácia = 0.46
12 accurácia = 0.49
13 accurácia = 0.47
14 accurácia = 0.47
15 accurácia = 0.46
16 accurácia = 0.42
17 accurácia = 0.39
18 accurácia = 0.37
19 accurácia = 0.36


O resultado mostra que para o conjunto de dados originais o melhor desempenho ocorre com um número de vizinhos igual a 1, valores de vizinhos iguais a 3 e 4 também oferecem bons resultados.

Como dito anteriormente algo que pode ser feito para melhorar o desempenho do modelo é a padronização de todas as variáveis. Padronizar no caso é fazer com todas as variáveis tenham valores entre 0 e 1. Assim, com todas as variáveis na mesma escala reduz-se o peso que uma variável possa ter sobre as outras. Este procedimento é realizado na célula a seguir. Foi utilizado o método MinMax que realiza o ajuste de escala a partir dos valores e máximo das séries. O método foi aplicado apenas as variáveis que apresentam valores máximos superiores a um como mencionado anteriormente.

In [76]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
X2 = X.copy()
X2['aspect_ratio'] = minmax.fit(X2[['aspect_ratio']]).transform(X2[['aspect_ratio']])
X2['lobedness'] = minmax.fit(X2[['lobedness']]).transform(X2[['lobedness']])
X2['lobedness'] = minmax.fit(X2[['lobedness']]).transform(X2[['lobedness']])
X2['entropy'] = minmax.fit(X2[['entropy']]).transform(X2[['entropy']])

x_train, x_test, y_train, y_test = train_test_split(X2, y, test_size=.3, random_state=1)

for k in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    y_predict = knn.predict(x_test)
    ac = accuracy_score(y_test, y_predict)
    print(f'{k} accurácia = {ac:.2f}')

1 accurácia = 0.62
2 accurácia = 0.56
3 accurácia = 0.59
4 accurácia = 0.52
5 accurácia = 0.55
6 accurácia = 0.59
7 accurácia = 0.56
8 accurácia = 0.51
9 accurácia = 0.48
10 accurácia = 0.46
11 accurácia = 0.45
12 accurácia = 0.44
13 accurácia = 0.48
14 accurácia = 0.48
15 accurácia = 0.44
16 accurácia = 0.43
17 accurácia = 0.43
18 accurácia = 0.40
19 accurácia = 0.41


Como visto ou uma pequena melhora no resultado do modelo após a padronização das variáveis.

* Você acredita que se realizar a seleção de atributos, o seu resultado será melhor? Justifique.

Sim. A seleção dos atributos pode gerar uma melhor resultado do modelo pode melhorar o desempenho do modelos pois, algunmas variáveis são mais correlacionadas entre si que outras. Assim, a redução do número de variáveis pode tanto melhorar o desempenho do modelo quanto otimizá-lo, já que uma quantidade menor de variáveis significa também um menor custo computacional. Algumas variáveis contam com variabilidade tão baixa que isto pode significar que não são determinantes para classificar as especies como por exemplo: smoothness, third_moment, uniformity.
Testemos a seguir o efeito que a remoção destas variáveis causa no desempenho do modelo

In [77]:
X.describe()

,eccentricity,aspect_ratio,elongation,solidity,stochastic_convexity,isoperimetric_factor,maximal_indentation_depth,lobedness,average_intensity,average_contrast,smoothness,third_moment,uniformity,entropy
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000
mean,0.719854,2.440210,0.513760,0.904158,0.943793,0.531234,0.037345,0.523845,0.051346,0.124535,0.017670,0.005928,0.000387,1.162630
std,0.208311,2.599043,0.195583,0.114639,0.115047,0.217532,0.038575,1.039639,0.035965,0.051860,0.013755,0.005294,0.000431,0.584854
min,0.117080,1.006600,0.107610,0.485490,0.396490,0.078376,0.002837,0.001464,0.005022,0.033415,0.001115,0.000229,0.000007,0.169400
25%,0.550623,1.211300,0.349623,0.890667,0.966230,0.346818,0.009521,0.016500,0.022843,0.083362,0.006901,0.002080,0.000102,0.718900
50%,0.763450,1.570750,0.501855,0.948130,0.992980,0.579160,0.023860,0.103615,0.042087,0.119375,0.014050,0.004447,0.000239,1.077450
75%,0.895097,2.343100,0.633373,0.976897,1.000000,0.700713,0.047834,0.416432,0.073046,0.163795,0.026127,0.008307,0.000516,1.554575
max,0.998710,19.038000,0.948340,0.993880,1.000000,0.858160,0.198980,7.206200,0.190670,0.280810,0.073089,0.029786,0.002936,2.708500


In [78]:
X3 = X2.drop(['smoothness', 'third_moment', 'uniformity'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X3, y, test_size=.3, random_state=1)

for k in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    y_predict = knn.predict(x_test)
    ac = accuracy_score(y_test, y_predict)
    print(f'{k} accurácia = {ac:.2f}')

1 accurácia = 0.62
2 accurácia = 0.56
3 accurácia = 0.59
4 accurácia = 0.52
5 accurácia = 0.55
6 accurácia = 0.59
7 accurácia = 0.56
8 accurácia = 0.51
9 accurácia = 0.48
10 accurácia = 0.47
11 accurácia = 0.45
12 accurácia = 0.43
13 accurácia = 0.48
14 accurácia = 0.48
15 accurácia = 0.44
16 accurácia = 0.43
17 accurácia = 0.43
18 accurácia = 0.40
19 accurácia = 0.40


Como esperado a remoção das variáveis referidas anteriormente praticamente não mudou o desempenho do modelo. Isto prova que uma análise mais detalhada poderá identificar as variáveis mais relevantes para o desempenho do modelo utilizado.